# *Modelo de Optimización para la Demanda de Queso Costeño*

El modelo de optimización presentado está diseñado para gestionar la demanda de queso costeño a través de una plataforma de marketplace, utilizando múltiples centros de acopio. Su objetivo principal es minimizar los costos asociados con el cumplimiento de la demanda del cliente, considerando los costos de transporte, tiempos de alistamiento y producción potencial en cada centro. El modelo busca reducir los costos totales de distribución en un sistema complejo que involucra un centro principal y varios centros secundarios, asegurando que se satisfaga la demanda total con la menor inversión en costos posible, mientras se cumplen las restricciones de capacidad y tiempo.

**Variables del Problema**

**N**: Número de centros de acopio.

**CAi**: Identificador del centro de acopio i para i=0…N donde i≠p. Estos centros de acopio complementan las unidades de producto de la demanda que el centro de acopio principal no tiene disponibles. Los centros de acopio **CAi** despachan hacia el centro de acopio principal.

**CAp**: Identificador del centro de acopio principal p (**p**∈[0,N] y **p**≠i). Este centro de acopio atiende directamente al cliente y es responsable de enviar la demanda completa al cliente.

**K(CAi)**: Cantidad del producto que se despacha desde el centro de acopio **CAi**, incluyendo unidades en stock y unidades potenciales que pueden estar listas en poco tiempo.

**Precio(CAi)**: Precio por kilo del producto despachado desde el centro de acopio **CAi**.

**cTransp(CAi)**: Costo del transporte del pedido desde el centro de acopio **CAi** a su destino.

**TiempoAlistam(CAi)**: Tiempo en horas para alistar el pedido desde el centro de acopio **CAi** a su destino.

**TiempoMaxDefinido**: Tiempo máximo definido para considerar la producción potencial.

**TiempoTransp(CAi)**: Tiempo de transporte desde el centro de acopio **CAi** a su destino.

**Tiempo(CAi)**: Tiempo total en horas para que el pedido llegue desde el centro de acopio **CAi** a su destino, sumando el tiempo de alistamiento y el tiempo de transporte.

**cTiempo**: Costo adicional por cada unidad de tiempo contemplado en la variable **Tiempo(CAi)**.

**Demanda**: Cantidad de producto solicitada por el cliente.

**Stock(CAi)**: Stock del producto en el centro de acopio **CAi**.

**Ppotencial(CAi)**: Cantidad de producto que potencialmente puede estar disponible en poco tiempo (potencial del día).

*Código desarrollado para la resolución del problema*

# IMPORTACIONES

In [40]:
from pyomo.environ import *
import pyomo.environ as pyo
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from tabulate import tabulate
from prettytable import PrettyTable
import openpyxl
import time

**demanda**: Cantidad de producto de acuerdo con la categoría del pedido a satisfacer.

**costo_transporte**: Costo por unidad de tiempo de espera para satisfacer la demanda.

**id**: Identificadores de Centro de Acopio (CAi).

**kg**: Stock del Producto en el Centro de Acopio (Stock(CAi)).

**produccion_potencial**: (Ppotencial(CAi)).

**costos_transporte**: (cTransp(CAi)).

**precio**: Precio del Producto por Kilo (Precio(CAi)).

**tiempos_transporte**:Tiempos de Transporte (TiempoTransp(CAi)).

**tiempo_alistamiento**:Tiempos de Alistamiento (TiempoAlistam(CAi)).

In [41]:
centros_acopio_df = pd.read_excel('data/centros_acopio.xlsx')
tiempos_transporte_df = pd.read_excel('data/tiempos_transporte.xlsx')
costos_transporte_df = pd.read_excel('data/costo_transporte.xlsx')

# REPRESENTACIÓN DE DATOS DE ENTRADA

In [42]:
centros_acopio = centros_acopio_df.copy()
headers = ["Id_CA", "Stock", "Ppotencial", "Precio", "CtranspCli", "TiempoTranspCli", "TiempoAlistam"]
table = [[row["Id_CA"], row["Stock"], row["Ppotencial"], row["Precio"], row["CtranspCli"], row["TiempoTranspCli"], row["TiempoAlistam"]] for _, row in centros_acopio.iterrows()]

print("Centros de Acopio:")
print(tabulate(table, headers=headers, tablefmt='grid'))
print()

costos_transporte_df.columns = [''] + list(costos_transporte_df.columns[1:])
tiempos_transporte_df.columns = [''] + list(tiempos_transporte_df.columns[1:])

print("Tabla 'Costos de Transporte':")
print(tabulate(costos_transporte_df, tablefmt='grid', showindex=False, headers='keys'))
print()

table = PrettyTable()
table.field_names = costos_transporte_df.columns.tolist()
for row in costos_transporte_df.values:
    table.add_row(list(row))

print("Tabla 'Tiempos de Transporte':")
print(tabulate(tiempos_transporte_df, tablefmt='grid', showindex=False, headers='keys'))
print()

table = PrettyTable()
table.field_names = tiempos_transporte_df.columns.tolist()
for row in tiempos_transporte_df.values:
    table.add_row(list(row))

print()

Centros de Acopio:
+---------+---------+--------------+----------+--------------+-------------------+-----------------+
| Id_CA   |   Stock |   Ppotencial |   Precio |   CtranspCli |   TiempoTranspCli |   TiempoAlistam |
+=========+=========+==============+==========+==============+===================+=================+
| CA1     |      10 |        0.5   |     5000 |        21841 |                37 |              10 |
+---------+---------+--------------+----------+--------------+-------------------+-----------------+
| CA2     |      12 |        0.625 |    10000 |        21180 |                42 |               9 |
+---------+---------+--------------+----------+--------------+-------------------+-----------------+
| CA3     |       4 |       14     |    15000 |        24132 |                32 |              47 |
+---------+---------+--------------+----------+--------------+-------------------+-----------------+
| CA4     |       8 |       13     |     2000 |        14215 |          

# DEMANDA DE QUESO

In [137]:
demanda_cliente = 1

# SELECCIÓN DEL CENTRO PRINCIPAL

La función evaluar_centros_acopio analiza diversos centros de acopio utilizando criterios de costos y tiempos de transporte para identificar el que mejor se ajuste a las necesidades logísticas establecidas. Selecciona la fila con la puntuación más baja, lo que indica que es el centro de acopio más eficiente en términos de costos y tiempos de entrega.

In [138]:
def evaluar_centros_acopio(centros_acopio_df, costos_transporte_df, tiempos_transporte_df):
    """
    Evalúa los centros de acopio y devuelve el mejor centro principal basado en costos y tiempos de transporte.
    
    Parameters:
        centros_acopio_df (DataFrame): DataFrame con información sobre centros de acopio.
        costos_transporte_df (DataFrame): DataFrame con costos de transporte.
        tiempos_transporte_df (DataFrame): DataFrame con tiempos de transporte.
    
    Returns:
        tuple: El identificador y el índice del mejor centro de acopio.
    """
    # Asegurar que los valores sean numéricos
    costos_transporte_df = costos_transporte_df.apply(pd.to_numeric, errors='coerce')
    tiempos_transporte_df = tiempos_transporte_df.apply(pd.to_numeric, errors='coerce')

    puntuaciones = []

    # Iterar sobre cada centro de acopio como posible centro principal
    for index_centro_principal, row_centro_principal in centros_acopio_df.iterrows():
        id_centro_principal = row_centro_principal['Id_CA']
        puntuacion_total = 0

        # Evaluar los costos y tiempos desde el centro principal hacia los otros centros
        for index, row in centros_acopio_df.iterrows():
            if index == index_centro_principal:
                continue  # Saltar el centro que está siendo evaluado como principal

            # Sumar costos y tiempos de transporte
            costo_transporte = costos_transporte_df.iloc[index_centro_principal, index]
            tiempo_transporte = tiempos_transporte_df.iloc[index_centro_principal, index]

            if pd.isna(costo_transporte) or pd.isna(tiempo_transporte):
                continue  # Saltar si hay valores nulos

            puntuacion_total += costo_transporte + tiempo_transporte

        # Agregar la puntuación total del centro evaluado
        puntuaciones.append((id_centro_principal, index_centro_principal, puntuacion_total))

    # Convertir a DataFrame para facilitar la selección
    df_puntuaciones = pd.DataFrame(puntuaciones, columns=['Id_CA', 'Index', 'Puntuacion'])

    # Seleccionar el centro con la puntuación mínima
    mejor_centro = df_puntuaciones.loc[df_puntuaciones['Puntuacion'].idxmin()]

    return mejor_centro['Id_CA'], mejor_centro['Index']

# Seleccionar el mejor centro de acopio sin pasar centro principal
centro_principal, centro_principal_index = evaluar_centros_acopio(centros_acopio_df, costos_transporte_df, tiempos_transporte_df)

print(f"El mejor centro de acopio seleccionado es: ID {centro_principal}, Índice {centro_principal_index}")

El mejor centro de acopio seleccionado es: ID CA2, Índice 1


In [139]:
def obtener_costos_y_tiempos(id_centro_acopio, centro_principal, costos_transporte_df, tiempos_transporte_df):
    """
    Obtiene el costo y tiempo de transporte desde el centro principal al centro especificado.
    
    Parameters:
        id_centro_acopio (str): Identificador del centro de acopio.
        centro_principal (str): Centro de acopio principal.
        costos_transporte_df (DataFrame): DataFrame con los costos de transporte entre centros.
        tiempos_transporte_df (DataFrame): DataFrame con los tiempos de transporte entre centros.

    Returns:
        (float, float): Costo y tiempo de transporte desde el centro principal.
    """
    if id_centro_acopio == centro_principal:
        return 0, 0  # Si es el centro principal, el costo y tiempo son 0
    
    # Encontrar el índice del centro principal
    index_centro_principal = centros_acopio_df.loc[centros_acopio_df['Id_CA'] == centro_principal].index[0]
    
    # Obtener costos y tiempos de transporte
    costo_transporte = costos_transporte_df.at[index_centro_principal, id_centro_acopio]
    tiempo_transporte = tiempos_transporte_df.at[index_centro_principal, id_centro_acopio]
    
    return costo_transporte, tiempo_transporte

def agregar_fila_final(lista_filas, id_centro_acopio, cantidad, precio, costo_transporte, tiempo_transporte, tiempo_alistamiento):
    """
    Agrega una fila a la lista de datos finales con los valores calculados.

    Parameters:
        lista_filas (list): Lista para almacenar las filas del DataFrame final.
        id_centro_acopio (str): Identificador del centro de acopio.
        cantidad (int): Cantidad de producto.
        precio (float): Precio del producto.
        costo_transporte (float): Costo de transporte desde el centro principal.
        tiempo_transporte (float): Tiempo de transporte desde el centro principal.
        tiempo_alistamiento (float): Tiempo de alistamiento del producto.
    """
    lista_filas.append({
        'Id_CA': id_centro_acopio,
        'Cantidad': cantidad,
        'Precio': precio,
        'Ctransp': costo_transporte,
        'TiempoTransp': tiempo_transporte,
        'TiempoAlistam': tiempo_alistamiento
    })

def generar_dataframe_centros(centro_principal):
    """
    Genera el DataFrame final que contiene la información de los centros de acopio, 
    sus cantidades de producto, precios, costos y tiempos de transporte.
    
    Parameters:
        centro_principal (str): Identificador del centro de acopio principal.

    Returns:
        DataFrame: DataFrame con los datos de centros de acopio, incluyendo transporte y tiempos.
    """
    filas_finales = []
    
    for _, row in centros_acopio_df.iterrows():
        id_centro_acopio = row['Id_CA']
        precio = row['Precio']
        
        # Obtener costos y tiempos de transporte
        costo_transporte, tiempo_transporte = obtener_costos_y_tiempos(
            id_centro_acopio, centro_principal, costos_transporte_df, tiempos_transporte_df)
        
        # Fila para el stock disponible
        #cantidad_stock = 0 if id_centro_acopio == centro_principal else row['Stock']
        cantidad_stock = row['Stock'] if id_centro_acopio == centro_principal else row['Stock']
        agregar_fila_final(filas_finales, id_centro_acopio, cantidad_stock, precio, costo_transporte, tiempo_transporte, 0)  # Tiempo alistamiento es 0 para stock
        
        # Fila para el potencial de acopio
        agregar_fila_final(filas_finales, id_centro_acopio, row['Ppotencial'], precio, costo_transporte, tiempo_transporte, row['TiempoAlistam'])
    
    return pd.DataFrame(filas_finales)

def generar_dataframes_dinamicos(centro_principal):
    """
    Genera DataFrames dinámicos para costos y tiempos de transporte con base en el centro principal.
    
    Parameters:
        centro_principal (str): Identificador del centro de acopio principal.

    Returns:
        (DataFrame, DataFrame): DataFrames de costos y tiempos de transporte.
    """
    # Encontrar el índice del centro principal
    index_centro_principal = centros_acopio_df.loc[centros_acopio_df['Id_CA'] == centro_principal].index[0]
    
    # Crear DataFrames de costos y tiempos con base en el centro principal
    costos_df = costos_transporte_df.iloc[[index_centro_principal]].reset_index(drop=True)
    tiempos_df = tiempos_transporte_df.iloc[[index_centro_principal]].reset_index(drop=True)
    
    return costos_df, tiempos_df

# Generar el DataFrame final
centros_df = generar_dataframe_centros(centro_principal)


# Generar DataFrames dinámicos
costos_df, tiempos_df = generar_dataframes_dinamicos(centro_principal)

# Resultados finales
print("DataFrame de Centros de Acopio:")
print(centros_df)

print("\nDataFrame de Costos de Transporte:")
print(costos_df)

print("\nDataFrame de Tiempos de Transporte:")
print(tiempos_df)

DataFrame de Centros de Acopio:
   Id_CA  Cantidad  Precio  Ctransp  TiempoTransp  TiempoAlistam
0    CA1    10.000    5000    17071            35              0
1    CA1     0.500    5000    17071            35             10
2    CA2    12.000   10000        0             0              0
3    CA2     0.625   10000        0             0              9
4    CA3     4.000   15000    20339            44              0
5    CA3    14.000   15000    20339            44             47
6    CA4     8.000    2000    12897            17              0
7    CA4    13.000    2000    12897            17             41
8    CA5    18.000   30000    28898            43              0
9    CA5    15.000   30000    28898            43             42
10   CA6     6.000   50000    15196            45              0
11   CA6    14.000   50000    15196            45             16
12   CA7    12.000   15000    16108            22              0
13   CA7    13.000   15000    16108            22         

In [144]:
def minimize_distribution_costs(demanda_cliente, centro_principal_index, centros_acopio_df, centros_df):
    model = pyo.ConcreteModel()

    # Conjunto de centros
    num_centros = len(centros_df) // 2
    model.I = pyo.RangeSet(0, num_centros - 1)

    # Parámetros
    model.demanda = pyo.Param(initialize=demanda_cliente)
    model.costo_tiempo = pyo.Param(initialize=100, mutable=True, within=pyo.NonNegativeReals)
    model.CentroPrincipal = pyo.Param(initialize=centro_principal_index)

    # Separar stock y producción potencial
    stock = centros_df.iloc[::2].reset_index(drop=True)
    produccion_potencial = centros_df.iloc[1::2].reset_index(drop=True)

    # Inicialización de parámetros
    model.Stock = pyo.Param(model.I, initialize=stock['Cantidad'].to_dict())
    model.Ppotencial = pyo.Param(model.I, initialize=produccion_potencial['Cantidad'].to_dict())
    model.Precio = pyo.Param(model.I, initialize=stock['Precio'].to_dict())
    model.CTransp = pyo.Param(model.I, initialize=stock['Ctransp'].to_dict())
    model.TiempoTransp = pyo.Param(model.I, initialize=stock['TiempoTransp'].to_dict())
    model.TiempoAlistam = pyo.Param(model.I, initialize=produccion_potencial['TiempoAlistam'].to_dict())
    model.KCAp = pyo.Param(initialize=centros_acopio_df.iloc[centro_principal_index]['Stock'])
    model.CostoTranspCli = pyo.Param(model.I, initialize=centros_acopio_df['CtranspCli'].to_dict())
    model.TiempoTranspCli = pyo.Param(model.I, initialize=centros_acopio_df['TiempoTranspCli'].to_dict())

    # Variables de decisión
    model.X = pyo.Var(model.I, within=pyo.NonNegativeReals)

    def objective_rule(model):
        total_cost = 0
        
        # Sumar los costos de los centros alternativos
        for i in model.I:
            # Costos variables: proporcionales a la cantidad transportada
            costo_producto = model.X[i] * model.Precio[i]
            costo_transporte = model.X[i] * model.CTransp[i] / max(1, model.X[i])  # Proporcional al transporte
            costo_tiempo = model.X[i] * (model.TiempoTransp[i] + model.TiempoAlistam[i]) * model.costo_tiempo
            
            total_cost += costo_producto + costo_transporte + costo_tiempo
        
        # Agregar los costos fijos del centro principal
        centro_principal = model.CentroPrincipal
        total_cost += (
            model.KCAp * model.Precio[centro_principal] +  # Costo de almacenamiento
            model.CostoTranspCli[centro_principal] +  # Costo fijo de transporte al cliente
            model.TiempoTranspCli[centro_principal] * model.costo_tiempo  # Costo del tiempo de entrega
        )
        
        return total_cost

    
    model.PrioridadCentroPrincipal = pyo.Constraint(expr=model.X[model.CentroPrincipal] == min(model.Stock[model.CentroPrincipal] + model.Ppotencial[model.CentroPrincipal], model.demanda))
    # Restricción de Satisfacción de Demanda
    model.DemandaSatisfecha = pyo.Constraint(expr=sum(model.X[i] for i in model.I) >= model.demanda)
    # Cantidad total distribuida <= stock + producción potencial
    model.CombinedStockPotential = pyo.Constraint(model.I, rule=lambda model, i: model.X[i] <= model.Stock[i] + model.Ppotencial[i])
    # Consumir primero el stock antes de usar la producción potencial (flexibilizado)
    model.ConsumoStockPrimero = pyo.Constraint(model.I, rule=lambda model, i: model.X[i] <= model.Stock[i] + model.Ppotencial[i])
    # Balance de transporte entre centros (ajuste para mayor flexibilidad)
    model.BalanceTransport = pyo.Constraint(model.I, rule=lambda model, i: sum(model.Y[j, i] for j in model.I) == model.X[i] if i != model.CentroPrincipal else pyo.Constraint.Skip)
    # Restricción de despacho desde otros centros (ajustado para mayor flexibilidad)
    model.RestriccionDespachoOtrosCentros = pyo.Constraint(rule=lambda model: sum(model.X[i] for i in model.I if i != model.CentroPrincipal) <= model.demanda - model.X[model.CentroPrincipal])


    # Resolución del modelo
    solver = pyo.SolverFactory('glpk')
    results = solver.solve(model, tee=True)
    
    # Manejo de casos donde no hay solución óptima
    if results.solver.termination_condition != pyo.TerminationCondition.optimal:
        print(f"El modelo no se resolvió de manera óptima. Estado: {results.solver.termination_condition}")
        print("Posibles razones: restricciones demasiado estrictas, falta de stock o demanda demasiado alta.")
        return None
    
    # Información del centro principal
    print("\n===============================================")
    print("       Información del Centro Principal  ")
    print("==============================================")
    try:
        centro_principal = centros_df.iloc[centro_principal_index + 1]
        print(f"\nCentro Principal: {centro_principal['Id_CA']}")
        print(f"Kilos asignados en el Centro Principal: {pyo.value(model.X[model.CentroPrincipal])}")
        print(f"Precio en el Centro Principal: {model.Precio[model.CentroPrincipal]}")
        print(f"Costo del Centro principal al cliente: {model.CostoTranspCli[model.CentroPrincipal]}")
        print(f"Tiempo del Centro principal al cliente: {model.TiempoTranspCli[model.CentroPrincipal]} horas")
    except Exception as e:
        print(f"Error al obtener información del centro principal: {e}")
    
    # Centros seleccionados para la operación y sus aportes
    print("\n===========================================================================")
    print("                   Centros Seleccionados para la Operación  ")
    print("===========================================================================")
    for i in model.I:
        cantidad_asignada = pyo.value(model.X[i])
        if cantidad_asignada > 0:
            centro_info = centros_df.iloc[i * 2]
            stock_disponible = model.Stock[i]
            produccion_disponible = model.Ppotencial[i]
            tiempo_alistamiento = model.TiempoAlistam[i]
            Precio = model.Precio[i]
            
            # Cálculo de cuánto se toma del stock y cuánto de la producción potencial
            cantidad_del_stock = min(cantidad_asignada, stock_disponible)
            cantidad_de_la_produccion = cantidad_asignada - cantidad_del_stock
            
            print(f"Centro: {centro_info['Id_CA']}")
            print(f"  - Precio en el centro: {Precio}")
            print(f"  - Kilos Asignados: {cantidad_asignada}")
            print(f"  - De los cuales:")
            print(f"    - {cantidad_del_stock} kg provienen del stock")
            print(f"    - {cantidad_de_la_produccion} kg provienen de la producción potencial")
            
            if i != centro_principal_index:
                print(f"  - Tiempo de alistamiento: {tiempo_alistamiento}")
                print(f"  - Costo de Transporte del Centro {centro_info['Id_CA']} al\n    Centro principal CA{centro_principal_index + 1}: ${model.CTransp[i]:,.2f}")
                print(f"  - Tiempo de Transporte del Centro {centro_info['Id_CA']} al\n    Centro Principal CA{centro_principal_index + 1}: {model.TiempoTransp[i]} horas\n\n")
            else:
                print("\n\n")
    
    return pyo.value(model.CostoTotal)



In [145]:
costo_total = minimize_distribution_costs(
    demanda_cliente=demanda_cliente,
    centro_principal_index=centro_principal_index,
    centros_acopio_df=centros_acopio_df,
    centros_df=centros_df
)

print("=======================================")
print(f"  Costo Total Minimizado: ${costo_total:,.2f}")
print("=======================================")

ERROR: Rule failed when generating expression for Constraint BalanceTransport
with index 0: AttributeError: 'ConcreteModel' object has no attribute 'Y'
ERROR: Constructing component 'BalanceTransport' from data=None failed:
        AttributeError: 'ConcreteModel' object has no attribute 'Y'


AttributeError: 'ConcreteModel' object has no attribute 'Y'